## 1. Setup and Imports

In [ ]:
import sys
import os
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.display import display, Image

# Import our modules
from database import init_db, get_db_connection
from models import Student, Course, Attendance, Settings
from face_recognition_module import FaceRecognitionSystem, process_student_images
from export_utils import export_attendance_to_excel, export_student_attendance_summary

print("✅ All modules imported successfully!")

## 2. Initialize Database

In [ ]:
# Initialize the database
init_db()
print("✅ Database initialized!")

## 3. Add Sample Courses

In [ ]:
# Add sample courses
sample_courses = [
    ('CS101', 'Introduction to Computer Science', 'Dr. Alice Johnson', 'Mon/Wed 10:00-11:30'),
    ('MATH201', 'Advanced Mathematics', 'Prof. Bob Smith', 'Tue/Thu 14:00-15:30'),
    ('PHY101', 'Physics Fundamentals', 'Dr. Carol White', 'Mon/Wed/Fri 9:00-10:00'),
]

for course_code, course_name, instructor, schedule in sample_courses:
    success = Course.add_course(course_code, course_name, instructor, schedule)
    if success:
        print(f"✅ Added course: {course_code} - {course_name}")
    else:
        print(f"⚠️ Course {course_code} already exists")

# Display all courses
courses = Course.get_all_courses()
df_courses = pd.DataFrame(courses, columns=['ID', 'Code', 'Name', 'Instructor', 'Schedule', 'Created'])
print("\nAll Courses:")
display(df_courses)

## 4. Add Students with Face Recognition

**Note:** You need to have student images in the `student_images` folder.

### Example: Add a student
Place a photo of the student in `student_images/` folder and run the cell below:

In [ ]:
# Example: Add a student
# Replace with actual student information and image path

student_info = {
    'student_id': 'STU001',
    'name': 'John Doe',
    'email': 'john.doe@university.edu',
    'phone': '123-456-7890',
    'image_path': 'student_images/john_doe.jpg'  # Make sure this file exists
}

# Check if image exists
if os.path.exists(student_info['image_path']):
    success = process_student_images(
        student_info['student_id'],
        student_info['name'],
        student_info['email'],
        student_info['phone'],
        student_info['image_path']
    )
    
    if success:
        print(f"✅ Student {student_info['name']} added successfully!")
    else:
        print(f"❌ Failed to add student {student_info['name']}")
else:
    print(f"⚠️ Image not found: {student_info['image_path']}")
    print("Please add student images to the 'student_images' folder first.")

## 5. View All Students

In [ ]:
# Display all students
students = Student.get_all_students()

if students:
    df_students = pd.DataFrame(students, columns=['ID', 'Student_ID', 'Name', 'Email', 'Phone', 'Image_Path', 'Face_Encoding', 'Created'])
    # Don't display the face encoding (it's binary data)
    df_students_display = df_students.drop('Face_Encoding', axis=1)
    print("All Students:")
    display(df_students_display)
else:
    print("No students found. Add students using the cell above.")

## 6. Test Face Recognition from Webcam

This will capture an image from your webcam and try to recognize the face.

In [ ]:
# Initialize face recognition system
fr_system = FaceRecognitionSystem()

# Capture from webcam
print("📸 Opening webcam... Press SPACE to capture, ESC to cancel")

camera = cv2.VideoCapture(0)

if not camera.isOpened():
    print("❌ Cannot open webcam")
else:
    captured = False
    
    while True:
        ret, frame = camera.read()
        
        if not ret:
            print("Failed to grab frame")
            break
        
        # Try to recognize face
        student_id, confidence, face_location = fr_system.recognize_face_from_frame(frame)
        
        if student_id and confidence > 0.5:
            frame = fr_system.draw_face_box(frame, face_location, student_id, confidence)
        
        # Display
        cv2.imshow('Face Recognition Test - Press SPACE to capture, ESC to cancel', frame)
        
        key = cv2.waitKey(1)
        
        if key == 32:  # SPACE key
            if student_id and confidence > 0.5:
                student = Student.get_student_by_id(student_id)
                print(f"\n✅ Recognized: {student['name']} ({student_id})")
                print(f"   Confidence: {confidence:.2%}")
                captured = True
            else:
                print("\n❌ No face recognized or confidence too low")
            break
        elif key == 27:  # ESC key
            print("\n❌ Cancelled")
            break
    
    camera.release()
    cv2.destroyAllWindows()
    
    print("\n✅ Camera closed")

## 7. Mark Attendance

In [ ]:
# Mark attendance for a student
student_id = 'STU001'  # Replace with actual student ID
course_code = 'CS101'  # Replace with actual course code

success = Attendance.mark_attendance(student_id, course_code)

if success:
    student = Student.get_student_by_id(student_id)
    course = Course.get_course_by_code(course_code)
    print(f"✅ Attendance marked for {student['name']} in {course['course_name']}")
else:
    print(f"⚠️ Attendance already marked for today or student/course not found")

## 8. View Attendance Records

In [ ]:
# Get all attendance records
attendance_records = Attendance.get_all_attendance()

if attendance_records:
    df_attendance = pd.DataFrame(attendance_records)
    print("All Attendance Records:")
    display(df_attendance)
else:
    print("No attendance records found.")

## 9. Calculate Attendance Percentage

In [ ]:
# Calculate attendance percentage for each student in each course
students = Student.get_all_students()
courses = Course.get_all_courses()

if students and courses:
    data = []
    
    for student in students:
        row = {
            'Student ID': student['student_id'],
            'Name': student['name']
        }
        
        for course in courses:
            percentage = Attendance.get_student_attendance_percentage(
                student['student_id'], 
                course['course_code']
            )
            row[course['course_code']] = f"{percentage}%"
        
        data.append(row)
    
    df_percentages = pd.DataFrame(data)
    print("Attendance Percentages:")
    display(df_percentages)
else:
    print("No students or courses found.")

## 10. Visualize Attendance Statistics

In [ ]:
# Visualize attendance for a specific student
student_id = 'STU001'  # Replace with actual student ID

student = Student.get_student_by_id(student_id)
courses = Course.get_all_courses()

if student and courses:
    course_codes = []
    percentages = []
    
    for course in courses:
        percentage = Attendance.get_student_attendance_percentage(
            student_id, 
            course['course_code']
        )
        course_codes.append(course['course_code'])
        percentages.append(percentage)
    
    # Create bar chart
    plt.figure(figsize=(10, 6))
    bars = plt.bar(course_codes, percentages, color=['green' if p >= 75 else 'red' for p in percentages])
    plt.axhline(y=75, color='orange', linestyle='--', label='Minimum Threshold (75%)')
    plt.xlabel('Course Code')
    plt.ylabel('Attendance Percentage (%)')
    plt.title(f'Attendance Overview for {student["name"]} ({student_id})')
    plt.ylim(0, 100)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, percentage in zip(bars, percentages):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{percentage:.1f}%',
                ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
else:
    print(f"Student {student_id} not found or no courses available.")

## 11. Export Attendance to Excel

In [ ]:
# Export all attendance records
filename = export_attendance_to_excel()

if filename:
    print(f"✅ Attendance exported to: {filename}")
else:
    print("❌ No data to export")

## 12. Export Attendance Summary

In [ ]:
# Export attendance summary with percentages
filename = export_student_attendance_summary()

if filename:
    print(f"✅ Attendance summary exported to: {filename}")
else:
    print("❌ No data to export")

## 13. Settings Management

In [ ]:
# Get current minimum attendance threshold
current_threshold = Settings.get_min_attendance_percentage()
print(f"Current minimum attendance threshold: {current_threshold}%")

# Update threshold (optional)
# Settings.set_min_attendance_percentage(80)
# print("Threshold updated to 80%")

## 14. Database Statistics

In [ ]:
# Get overall statistics
total_students = len(Student.get_all_students())
total_courses = len(Course.get_all_courses())
total_attendance = len(Attendance.get_all_attendance())

print("📊 System Statistics")
print("=" * 40)
print(f"Total Students: {total_students}")
print(f"Total Courses: {total_courses}")
print(f"Total Attendance Records: {total_attendance}")
print("=" * 40)

# Create a pie chart
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Overall stats
labels = ['Students', 'Courses', 'Attendance Records']
sizes = [total_students, total_courses, total_attendance]
colors = ['#667eea', '#764ba2', '#f093fb']

ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax1.set_title('System Overview')

# Attendance by course
courses = Course.get_all_courses()
if courses:
    course_names = []
    course_attendance = []
    
    for course in courses:
        records = Attendance.get_attendance_by_course(course['course_code'])
        course_names.append(course['course_code'])
        course_attendance.append(len(records))
    
    ax2.bar(course_names, course_attendance, color=colors[0])
    ax2.set_xlabel('Course Code')
    ax2.set_ylabel('Number of Attendance Records')
    ax2.set_title('Attendance by Course')
    ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## Conclusion

This notebook demonstrated all the key features of the Face Recognition Attendance System:

✅ Database initialization and management  
✅ Student and course management  
✅ Face recognition and training  
✅ Attendance marking and tracking  
✅ Attendance percentage calculation  
✅ Data visualization  
✅ Excel export functionality  

### Next Steps:

1. Run the Flask web application: `python app.py`
2. Access the admin panel at: `http://localhost:5000/admin/login`
3. Mark attendance at: `http://localhost:5000/mark-attendance`

For questions or issues, refer to the README.md file.